In [ ]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

from calc_diff_obs_model import *
from plt_spaghetti_scatter import *

In [ ]:
year = 2016
month = 12
tid = '00'
#tid = '18'
station = 'Haukeliseter'
savefig = 0 # 1 = yes, 0 = no

In [ ]:
#### days 
t = ['20', '21', '22', '23', '24', '25', '26', '27']


In [ ]:
#var = 'PP'
#var = 'SP'
#var = 'T2'
#var = 'WD'
#var = 'WS'
if var == 'PP':
    var_name = 'precipitation_amount_acc'
elif var == 'SP':
    var_name = 'air_pressure_at_sea_level'
elif var == 'T2':
    var_name = 'air_temperature_2m'
elif var == 'WD':
    var_name = 'wind_direction'
elif var == 'WS':
    var_name = 'wind_speed'


figdir_var = '../../Figures/MEPS/%s' %(var_name)
figdir_scatter = '../../Figures/MEPS/scatter_MEPS_obs/%s' %(var_name)
if savefig == 1:
    cF.createFolder('%s/' %(figdir_var))
    cF.createFolder('%s/' %(figdir_scatter))



form = 'png'

In [ ]:
### MEPS
if var == 'SP' or var == 'PP':
    level = 'sfc'
elif var == 'T2':
    level = '2m'
elif var == 'WD' or var == 'WS':
    level = '10m'

ncdir_var = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)



################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Haukeli_obs_Dec2016'

In [ ]:
# read in the Haukeliseter time file, sorted by daily values
Haukeli_Date = pd.read_csv('%s/Observations-Date-Hour.txt' %(txtdir),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')
DateHour = Haukeli_Date['Date-Hour(UTC)']

In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir,txt_filename),\
                    sep = ';',header=18, skipfooter = 2, engine= 'python')

if var == 'PP':
    obsvar = Haukeli['RR_1'].astype(float)         # Precipitation (1 hour); [mm]
elif var == 'T2':
    obsvar = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
elif var == 'SP':
    obsvar = Haukeli['PR'].astype(float)           # PR;Air pressure at sea level;hPa
elif var == 'WD' or var == 'WS':
    obsvar_DD = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
    obsvar_FF = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

if var == 'PP' or var == 'T2' or var == 'SP':
    obsvar0, dt_obsvar0, hour_obsvar0, day_obsvar0, month_obsvar0, year_obsvar0 = obsDat.valid_values(obsvar, DateHour)
elif var == 'WD' or var == 'WS':
    idxFF = np.where(obsvar_FF[:] == -9999)
    idxDD = np.where(obsvar_DD[:] == -9999)

    obsvar_FF[idxFF[0][:]] = np.nan
    obsvar_DD[idxDD[0][:]] = np.nan

    ### calculate the U, V wind component for barb plot
    # http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html
    # first calculate the mathematical wind direction in deg
    md_deg = 270 - obsvar_DD
    for k in range(0,md_deg.shape[0]):
        if md_deg[k] <0 :
            md_deg[k] = md_deg[k] +360
    md_rad = math.pi/180. * md_deg
    uwind = obsvar_FF*np.cos(md_rad)
    vwind = obsvar_FF*np.sin(md_rad)
    
    obsvar_FF_wind, obsvar_DD_wind, \
            obsvar_dt_wind, obsvar_hour_wind, \
            obsvar_day_wind, obsvar_month_wind, \
            obsvar_year_wind = obsDat.valid_values_wind(obsvar_FF, obsvar_DD, DateHour)
    if var == 'WD':
        obsvar0 = obsvar_DD_wind
    elif var == 'WS':
        obsvar0 = obsvar_FF_wind

In [ ]:
fn_var = dict()

if var == 'PP' or var == 'T2' or var == 'SP':
    model_var = dict()
elif var == 'WD' or var == 'WS':
    model_var_x = dict()
    model_var_y = dict()
    
    model_var = dict()


time_var      = dict()
lead_time_var = dict()
ini_day       = dict()
calday        = dict()
calmon        = dict()


In [ ]:
calendar_day = []
initialisation_day = []
calendar_month = []
calendar_year = []
initialisation_hh = []

In [ ]:
diff_var = dict()
Diff_var_0 = []
Diff_var_1 = []
Diff_var_2 = []
Diff_var_3 = []
Diff_var_4 = []
Diff_var_5 = []
Diff_var_6 = []
Diff_var_7 = []
Diff_var_8 = []
Diff_var_9 = []


all_day_max_var = []
all_day_min_var = []

In [ ]:
model_var_0 = []
model_var_1 = []
model_var_2 = []
model_var_3 = []
model_var_4 = []
model_var_5 = []
model_var_6 = []
model_var_7 = []
model_var_8 = []
model_var_9 = []

obs_var = []

In [ ]:
#t = ['24']
for day in t:
    
    if var == 'T2' or var == 'SP' or var == 'WD' or var == 'WS':
        ### connect variable data for two days
        df1 = obsvar0[int(day)-1]
        df2 = obsvar0[int(day)]
        Variable_obs = np.concatenate((df1,df2),axis=0)
        df3 = obsvar0[int(day)+1][0]
        Variable_obs = np.append(Variable_obs,df3)
    



    
    
    for ens_memb in range(0,10):
        fn_var[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_var,year,month,day,tid,ens_memb))
        
        time_var[ens_memb] = fn_var[ens_memb].variables['time']
        time_var[ens_memb] = fv.fill_nan(time_var[ens_memb][:])
        lead_time_var[ens_memb] = np.arange(0,time_var[ens_memb].shape[0])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_var[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_var[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)        
# Read in the variable   
        if var == 'PP' or var == 'T2' or var == 'SP':
            model_var[ens_memb] = fn_var[ens_memb].variables[var_name][:]
            if var == 'T2':
                model_var[ens_memb] = model_var[ens_memb]-273.15
            elif var == 'SP':
                model_var[ens_memb] = model_var[ens_memb]/100
            #substitute missing values with nan 
            model_var[ens_memb] = fv.fill_nan(model_var[ens_memb][:])

          

        elif var == 'WD' or var == 'WS':
            model_var_x[ens_memb] = fn_var[ens_memb].variables['x_wind_10m'][:]
            model_var_y[ens_memb] = fn_var[ens_memb].variables['y_wind_10m'][:]
            #substitute missing values with nan 
            model_var_x[ens_memb] = fv.fill_nan(model_var_x[ens_memb][:,:])
            model_var_y[ens_memb] = fv.fill_nan(model_var_y[ens_memb][:,:])


            #### calculate wind direction / speed
            if var == 'WD':
                model_var[ens_memb] = np.arctan2(model_var_y[ens_memb][:,0], model_var_x[ens_memb][:,0])
                model_var[ens_memb] = 270-np.rad2deg(model_var[ens_memb])  # convert from math to meteo direction and to degrees
                larger360 = np.where(model_var[ens_memb] >360.)
                model_var[ens_memb][larger360] = model_var[ens_memb][larger360]-360
            elif var == 'WS':
                model_var[ens_memb] = np.sqrt((model_var_x[ens_memb][:,0])**2 + (model_var_y[ens_memb][:,0])**2) 

#### calculate every 60 min the accumulation over 48 hours
    if var == 'PP':
        Variable_obs = acc.accumulation_dt60_for48h(obsvar0, ini_day)
################

#### calculate the ensemble mean of variable every 3 hours (all 10 ensemble member averaged)
    val = np.zeros(shape=(10,np.asarray(model_var[0])[::3].shape[0]))
    model_var_std = []
    model_var_mean = []
    for ens_memb in range(0,10):
        if var == 'PP' or var == 'T2' or var == 'SP':
            val[ens_memb] = np.asarray(model_var[ens_memb])[::3,0]
        elif var == 'WD' or var == 'WS':
            val[ens_memb] = np.asarray(model_var[ens_memb])[::3]
          
    ### std, mean
    model_var_std.append(np.nanstd(val,axis=0,ddof=1))
    model_var_mean.append(np.mean(val,axis=0))
    
    time_EM_mean = lead_time_var[0][0:np.asarray(model_var[0]).shape[0]:3]

#    time_model_var_mean = lead_time_var[0][~np.isnan(model_var_mean).any(axis=0)]
 #   model_var_mean = np.asarray(model_var_mean)[~np.isnan(model_var_mean)]
################    

### save calendar day etc. for scatter plot
    calendar_day.append(calday)
    initialisation_day.append(ini_day)
    calendar_month.append(calmon)
    calendar_year.append(year)
    initialisation_hh.append(hh) 
########


#### calculate the difference between obs and model
    diff_var, Diff_var_0, \
        Diff_var_1, Diff_var_2, Diff_var_3, \
        Diff_var_4, Diff_var_5, Diff_var_6, \
        Diff_var_7, Diff_var_8, Diff_var_9  = calc_diff(diff_var, \
                                                       model_var, Variable_obs, Diff_var_0,\
                                                       Diff_var_1, Diff_var_2, Diff_var_3, \
                                                       Diff_var_4, Diff_var_5, Diff_var_6, \
                                                       Diff_var_7, Diff_var_8, Diff_var_9,var=var)

    if var == 'WD':
        for ens_memb in range(0,10):
            if (diff_var[ens_memb] < -180).any() == True:
                diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)] = \
                diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)]+360
                
            elif (diff_var[ens_memb] > 180).any() == True:
                diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)] = \
                    diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)]-360

#########################################
### save daily variable
    if var == 'PP' or var == 'T2' or var == 'SP':
        model_var_0.append(model_var[0][:,0])
        model_var_1.append(model_var[1][:,0])
        model_var_2.append(model_var[2][:,0])
        model_var_3.append(model_var[3][:,0])
        model_var_4.append(model_var[4][:,0])
        model_var_5.append(model_var[5][:,0])
        model_var_6.append(model_var[6][:,0])
        model_var_7.append(model_var[7][:,0])
        model_var_8.append(model_var[8][:,0])
        model_var_9.append(model_var[9][:,0])
    elif var == 'WD' or var == 'WS':
        model_var_0.append(model_var[0])#[:,0])
        model_var_1.append(model_var[1])#[:,0])
        model_var_2.append(model_var[2])#[:,0])
        model_var_3.append(model_var[3])#[:,0])
        model_var_4.append(model_var[4])#[:,0])
        model_var_5.append(model_var[5])#[:,0])
        model_var_6.append(model_var[6])#[:,0])
        model_var_7.append(model_var[7])#[:,0])
        model_var_8.append(model_var[8])#[:,0])
        model_var_9.append(model_var[9])#[:,0])
    
    obs_var.append(Variable_obs)
    
#########################################    
    if hh < 10:
        hh = '0%s' %hh
    else:
        hh = hh
#########################################
    for ens_memb in range(0,10):
        if var == 'PP' or var == 'T2' or var == 'SP':
            lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb][:,0])]
        elif var == 'WD' or var == 'WS':
            lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb])]
        model_var[ens_memb]     = model_var[ens_memb][~np.isnan(model_var[ens_memb])]
    
    
    
        fn_var[ens_memb].close()
###########

        
        
        

##### PLOTTING ############################################
    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    xdays = ['%s-%s-%s' %(year,month,ini_day), '',
        6,'',12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+1), '',
        6,'', 12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+2)]
####### plot variable    
    lgd = plt_variable(lead_time_var,model_var,Variable_obs,time_EM_mean, np.asarray(model_var_mean)[0,:],var=var,
                       xdays=xdays, title = title)
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_var, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_var,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

###################################################################    
#### plot scatter observation vs. model
    if day == '20':
        cc = no1
    elif day == '21':
        cc = no2
    elif day == '22':
        cc = no3
    elif day == '23':
        cc = no4
    elif day == '24':
        cc = no5
    elif day == '25':
        cc = no6
    elif day == '26':
        cc = no7
    label = '%s, %s %s %s %s UTC' %(calday,ini_day, 
                                        calmon,year,hh)
    plt_scatter_obs_model(Variable_obs, model_var,cc,var=var,label=label)  
    if savefig == 1:
        plt.savefig('%s/obs_model_%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/obs_model_%s' %(figdir_scatter,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
     

###################################################################    
#### plot scatter difference
    plt_scatter_diff(diff_var, lead_time_var, Variable_obs, cc, var=var,label=label)
    if savefig == 1:
        plt.savefig('%s/diff_%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/diff_%s' %(figdir_scatter,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

In [ ]:
###################################
model_var_all_day = dict()

model_var_all_day[0]=(model_var_0)
model_var_all_day[1]=(model_var_1)
model_var_all_day[2]=(model_var_2)
model_var_all_day[3]=(model_var_3)
model_var_all_day[4]=(model_var_4)
model_var_all_day[5]=(model_var_5)
model_var_all_day[6]=(model_var_6)
model_var_all_day[7]=(model_var_7)
model_var_all_day[8]=(model_var_8)
model_var_all_day[9]=(model_var_9)

In [ ]:
###################################################################    
#### plot scatter observation vs. model
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(0,len(obs_var)-1),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

### 20.12. - 23.12. 
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(0,4),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

### 24.12. - 26.12.
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(4,len(obs_var)-1),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

In [ ]:
###################################################################    
#### plot scatter difference
Diff_obs_model_var = dict()

Diff_obs_model_var[0]=(Diff_var_0)
Diff_obs_model_var[1]=(Diff_var_1)
Diff_obs_model_var[2]=(Diff_var_2)
Diff_obs_model_var[3]=(Diff_var_3)
Diff_obs_model_var[4]=(Diff_var_4)
Diff_obs_model_var[5]=(Diff_var_5)
Diff_obs_model_var[6]=(Diff_var_6)
Diff_obs_model_var[7]=(Diff_var_7)
Diff_obs_model_var[8]=(Diff_var_8)
Diff_obs_model_var[9]=(Diff_var_9)

lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(0,len(Diff_obs_model_var[0])-1),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()


### 20.12. - 23.12. 
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(0,4),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

### 24.12. - 26.12.
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(4,len(Diff_obs_model_var[0])-1),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()